In [3]:
from hart_tools import (
    prep_manifest,
    read_cvr, 
    read_cvrs, 
    check_for_contest,
    filter_cvr_contest,
    tabulate_styles
)
# from consistent_sampling import (
#    assign_random_number,
#    get_ballots_threshold
#)

from IPython.core.interactiveshell import InteractiveShell
# from rilacs.misc import get_bravo_workload, get_data_dict, get_workload_from_mart, get_workloads
# from rilacs.plotting import plot_workload
# from rilacs.martingales import (
#     apriori_Kelly_martingale,
#     dKelly_martingale,
#     sqKelly_martingale,
# )
from assertion_audit_utils import \
    Assertion, Assorter, CVR, TestNonnegMean, check_audit_parameters, find_margins,\
    find_p_values, find_sample_size, new_sample_size, prep_sample, summarize_status,\
    write_audit_parameters

import os
import io
import re
import numpy as np
import math
import csv
import pandas as pd
import warnings
import copy
import xml.etree.ElementTree as ET
import xml.dom.minidom
import cryptorandom
from cryptorandom.cryptorandom import SHA256, int_from_hash_py3, int_from_hash
#from pandas.io.parsers import ParserError

# parse XMLs
#cvr_list = read_cvrs(cvr_folder = "files-for-testing-v2-copy")

In [16]:
cvr_path = "./Data/hart/OC2021/oc_cvrs-for-testing-v2-copy/
cvr_file = "2FL9iY1EIMx3QxOW4uK23O2jV72VApHd.xml"
#cvr_path = "files-for-testing-v2-copy/0a7EqyblZ6CMFQbkN0kkRykS5w40Wg8q.xml"
namespaces = {'xsi': "http://www.w3.org/2001/XMLSchema-instance",
              "xsd": "http://www.w3.org/2001/XMLSchema",
              "xmlns": "http://tempuri.org/CVRDesign.xsd"}
pat = re.compile('[^\w\-\<\>\[\]"\\\']+') # match "word" characters and hyphens
raw = io.open(cvr_path, mode="r", encoding="utf-8").read()
cleaned = re.sub(pat, ' ', raw)
#cvr_root = ET.parse(cleaned).getroot()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc9 in position 1601: invalid continuation byte

In [2]:
read_hart_cvr("files-for-testing-v2-copy/2FL9iY1EIMx3QxOW4uK23O2jV72VApHd.xml")
#read_hart_cvr("files_for_testing_v2/0G7r0rvjmnWNXzIG1VeGY6oezkVD82Gc.xml")
#read_hart_cvr("files_for_testing_v2/0Hr92V281Odmirlz7golaEz4cq6Qxsli.xml")

TypeError: expected string or bytes-like object

In [4]:
space = []
no_space = []
for i in cvr_list:
    space.append(check_for_contest(i, 'UNITED STATES REPRESENTATIVE\n46th District'))
    no_space.append(check_for_contest(i, 'UNITED STATES REPRESENTATIVE46th District'))
cvr_names = os.listdir("files_for_testing_v2")
#these CVR does not
#space
[i for i, x in enumerate(no_space) if x]
cvr_names[222], cvr_names[955]

[222, 955]

In [27]:
read_hart_cvr('cvrs_for_testing_v2/1QNcNBvTgTnyam1JOi7UxM575JvMDD8A.xml').votes.keys()
read_hart_cvr('cvrs_for_testing_v2/0G7r0rvjmnWNXzIG1VeGY6oezkVD82Gc.xml').votes.keys()

dict_keys(['PRESIDENT AND VICE PRESIDENT', 'UNITED STATES REPRESENTATIVE\n46th District', 'STATE SENATOR\n29th District', 'MEMBER OF THE STATE ASSEMBLY\n65th District', 'CITY OF ANAHEIM\nMember, City Council, District 1', 'Proposition 14', 'Proposition 15', 'Proposition 16', 'Proposition 17', 'Proposition 18', 'Proposition 19', 'Proposition 20', 'Proposition 21', 'Proposition 22', 'Proposition 23'])

In [2]:
print(cvr_list[0].id)
print(cvr_list[0].votes)

CVR.get_vote_from_cvr(contest = 'PRESIDENT AND VICE PRESIDENT', candidate = 'JOSEPH R. BIDEN\nKAMALA D. HARRIS', cvr = cvr_list[0])

51_1
{'PRESIDENT AND VICE PRESIDENT': {'JOSEPH R. BIDEN\nKAMALA D. HARRIS': True}, 'UNITED STATES REPRESENTATIVE\n45th District': {'KATIE PORTER': True}, 'STATE SENATOR\n37th District': {'DAVE MIN': True}, 'MEMBER OF THE STATE ASSEMBLY\n68th District': {'MELISSA FOX': True}, 'SOUTH ORANGE COUNTY COMMUNITY COLLEGE DISTRICT\nGoverning Board Member,\nTrustee Area 1': {'NA': True}, 'SOUTH ORANGE COUNTY COMMUNITY COLLEGE DISTRICT\nGoverning Board Member,\nTrustee Area 6': {'RYAN DACK': True}, 'SOUTH ORANGE COUNTY COMMUNITY COLLEGE DISTRICT\nGoverning Board Member,\nTrustee Area 7': {'NA': True}, 'SADDLEBACK VALLEY UNIFIED SCHOOL DISTRICT\nGoverning Board Member,\nTrustee Area 2': {'NA': True}, 'CITY OF LAKE FOREST\nMember, City Council, District 5': {'JEANNETTE J. NADAL': True}, 'Proposition 14': {'No': True}, 'Proposition 15': {'NA': True}, 'Proposition 16': {'Yes': True}, 'Proposition 17': {'NA': True}, 'Proposition 18': {'No': True}, 'Proposition 19': {'NA': True}, 'Proposition 20': {'No

True

In [3]:
# read in manifest
manifest = pd.read_csv("manifest-CARCL2021.csv")
manifest.head()

,Container,Tabulator,Batch Name,Number of Ballots
0,Mail,1,1,60
1,Mail,1,2,21
2,Mail,1,3,123
3,Mail,1,4,59
4,Mail,1,5,87


In [4]:
CVR.assign_sample_nums(cvr_list, prng = SHA256(1234567890))

True

In [5]:
def sort_cvr_list(cvr_list):
    cvr_list.sort(key=lambda x: x.sample_num) ## Use this to sort CVRs

In [30]:
### Inputs should be sorted_cvr_list and contest dict which has CVRs already sampled and sample size needed

## Contest dict will have the N_c and the big n_c, we will add to contest dict the threshold
## Also need to know previous threshold -- just actually keep this in the function
# or is confirmed flag
# look for max_p < risk_limit -> if so done with that contest
## Return 1 vector with the sample
## Add flag for whether or not recordding data from all contests even if not "sampled"
def get_ballots_threshold(sorted_cvr_list, sample_size_dict, sampled_CVRs = []):
    ## TO DO: Create dict of current sample size based on length of sampled_cvrs
    # ultimately will iterate over that
    
    ## find the percentile and then filter it
    # https://numpy.org/doc/stable/reference/generated/numpy.percentile.html
    
    # get list of contests to consider from threshold dict
    contest_list = list(sample_size_dict.keys())
    # get list of ticket numbers for the cvr list
    ticket_number_list = [cvr.sample_num for cvr in sorted_cvr_list]
    # loop through each contest
    for contest in contest_list:
        # return true if contest on ballot
        contest_on_ballot = [contest in cvr.votes.keys() for cvr in sorted_cvr_list]
        contest_indices = np.where(contest_on_ballot)[0]
        # get ballots where contest on ballot and still need to sample more ballots and append to sampled_CVRs if not already included
        sampled_ballots = 0
        for i in contest_indices:
            # while loop instead?
            if cvr_list[i] not in sampled_CVRs and sampled_ballots < sample_size_dict[contest]:
                sampled_CVRs.append(cvr_list[i])
                print(cvr_list[i].sample_num)
            sampled_ballots += 1
    # NOTE: SHOULD WE ADD ALSO WHICH CONTEST IT'S BEING SAMPLED FOR -- PART OF THE DICT???
    # return CVRs to sample
    return sampled_CVRs

In [6]:
cvr_list[0].id

'51_1'

In [33]:
contest = 'PRESIDENT AND VICE PRESIDENT'
sample_size_dict = {'PRESIDENT AND VICE PRESIDENT' : 30}
sampled_CVRs = []
# contest dict should be the input
contest_dict = {'PRESIDENT AND VICE PRESIDENT' : {'sample_size' : 30, 'sampled_cvrs' : []}}
get_ballots_threshold(cvr_list, sample_size_dict = {'PRESIDENT AND VICE PRESIDENT' : 30})

1026812401179559734840323878948447767126576087957207343008967393315095088472
1166903965375130243016819165255905408695426657718064991909449107187619759261
1265156423548541945017553016822968829157236421116862893949311175007690291413
1428178458846043300912263228186556896448346083708068924897081303841939859076
1445633562411753691538337623922072823479537674611532626600177217276309421250
1661148829561224073071512793708641314821871794153520464188578264354399282816
1815934151440754826680999534360079473422736539859653206255484998315077913035
1875985035713203744441933605650952645878955205090914073664977967961933446791
1945757082263622806349893892776584060671757459593031338666858162559309170691
1975905604551050479790309922520126672408198263074445701163010812699255413732
2023284194070044604782936782948516541127626636143230124159319188681723175868
2358156929514683509126047630170320542647088345149972791937210130395864382620
2505490589754435715994578281013395026672711055902569480601658754674787610617

In [5]:
# get unique styles
styles = tabulate_styles(cvr_list)

TypeError: 'CVR' object is not subscriptable

In [6]:
# get reported votes
vote_count_df = get_votes(cvr_list)

NameError: name 'get_votes' is not defined

In [12]:
contests = make_preliminary_contests_dict(vote_count_df, risk_limit = .05)

In [11]:
assertions = Assertion.make_all_assertions(contests)

In [14]:
find_margins(contests, assertions, cvr_list)

AttributeError: 'list' object has no attribute 'votes'

In [6]:
risk_function = "kaplan_kolmogorov" 
alpha = .05
contests = vote_count_df["contest"].unique()
error_rate = .001


#aggregate across styles to get sampling fraction to verify each contest
contest_totals_df = vote_count_df.groupby(["contest","vote"])["num_votes"].sum().reset_index()
sample_fractions = []
margins = []
#ballots = []
for i in range(len(contests)):
    valid_votes = sorted(contest_totals_df["num_votes"][(contest_totals_df["contest"] == contests[i]) & (contest_totals_df["vote"] != "NA")].tolist(), reverse = True)
    ballots_cast = sum(contest_totals_df["num_votes"][contest_totals_df["contest"] == contests[i]])
    #ballots = ballots.append(ballots_cast)
    #if there's only one ballot in the contest, check it.
    if ballots_cast == 1:
        sample_fractions.append(1)
        margins.append(0)
        continue
    #if there is only one option with valid votes, the next option received 0 (though we don't know what it is)
    if len(valid_votes) == 1:
        valid_votes.append(0)
    #Margins eventually need to be able to accomodate multiple winners
    m = (valid_votes[0] - valid_votes[1]) / ballots_cast
    margins.append(m)
    if m == 0:
        sample_fractions.append(1)
        continue
    #N might need to be more general to account for phantoms
    if risk_function == "kaplan_markov":
        risk_fn = lambda x: TestNonnegMean.kaplan_markov(x, g = .1)
    elif risk_function == "kaplan_wald":
        risk_fn = lambda x: TestNonnegMean.kaplan_wald(x, g = .1)
    elif risk_function == "kaplan_kolmogorov":
        risk_fn = lambda x: TestNonnegMean.kaplan_kolmogorov(x, N = ballots_cast, g = .1)
    elif risk_function == "kaplan_martingale":
        risk_fn = lambda x: TestNonnegMean.kaplan_martingale(x, N = ballots_cast, g = .1)[0]
    else:
        "Input a valid risk_function."

    sample_fractions.append(TestNonnegMean.initial_sample_size(risk_function = risk_fn, margin = m, N = ballots_cast, alpha = alpha, error_rate = error_rate, u = 1, t = 1/2) / ballots_cast)
#Uses S4, eventually we will want to use a more efficient method
dict(zip(contests, sample_fractions))

{'AA-City of Orange': 1.0,
 'ANAHEIM ELEMENTARY SCHOOL DISTRICT\nGoverning Board Member,\nTrustee Area 3': 1.0,
 'BB-City of San Clemente': 0.9166666666666666,
 'BREA OLINDA UNIFIED SCHOOL DISTRICT\nGoverning Board Member,\nTrustee Area 5': 1,
 'BUENA PARK LIBRARY DISTRICT\nTrustee': 1.0,
 'BUENA PARK LIBRARY DISTRICTTrustee': 1,
 'CAPISTRANO UNIFIED SCHOOL DISTRICT\nGoverning Board Member,\nTrustee Area 2': 0.9230769230769231,
 'CAPISTRANO UNIFIED SCHOOL DISTRICT\nGoverning Board Member,\nTrustee Area 3': 1.0,
 'CAPISTRANO UNIFIED SCHOOL DISTRICT\nGoverning Board Member,\nTrustee Area 5': 1,
 'CC-City of Tustin': 0.6428571428571429,
 'CITY OF ALISO VIEJO\nMember, City Council': 1,
 'CITY OF ANAHEIM\nMember, City Council, District 1': 1.0,
 'CITY OF ANAHEIM\nMember, City Council, District 4': 1.0,
 'CITY OF ANAHEIM\nMember, City Council, District 5': 1.0,
 'CITY OF ANAHEIMMember, City Council, District 5': 1,
 'CITY OF BREA\nCity Treasurer': 1.0,
 'CITY OF BREA\nMember, City Council': 

In [5]:
get_expected_sample_size(cvr_list, risk_function = "kaplan_kolmogorov", error_rate = 0, alpha = .05)

/Users/Jake/miniconda3/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


689.0428398005711

In [6]:
get_expected_sample_size(cvr_list, risk_function = "kaplan_kolmogorov", error_rate = .001, alpha = .05)

/Users/Jake/miniconda3/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


702.0676967683497

In [7]:
get_expected_sample_size(cvr_list, risk_function = "kaplan_markov", error_rate = 0, alpha = .05)

/Users/Jake/miniconda3/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


703.2841084058059

In [8]:
get_expected_sample_size(cvr_list, risk_function = "kaplan_markov", error_rate = .001, alpha = .05)

/Users/Jake/miniconda3/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


718.6366677980894

In [ ]:
# example return ballots
sorted_cvr_list = assign_random_number(cvr_list)
get_ballots_threshold(sorted_cvr_list, sample_size_dict = {'Z-City of Newport Beach' : 5, 
                                                    'UNITED STATES REPRESENTATIVE46th District' : 5},
                     sampled_CVRs = [])

In [31]:
N = 5000
alpha = 0.05
m_null = 1 / 2

true = np.concatenate((np.repeat(0, 990), np.repeat(1, 1010)))
reported = np.concatenate((np.repeat(0, 990), np.repeat(1, 1010)))
omega = reported - true
v = 2 * np.mean(reported) - 1
b = (1 - omega) / (2 - v)
mu_0 = 1/2

In [32]:
get_workloads(
    workload_dict = {"SqKelly": lambda x: get_workload_from_mart(
        x,
        mart_fn=lambda y: sqKelly_martingale(y, 1 / 2, N=N, D=20, beta=1),
        alpha=alpha,
    )},
    data = b
)

{'SqKelly': array([2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000])}

In [7]:
vote_count_df = votes_df
contests = vote_count_df["contest"].unique()
#aggregate across styles to get sampling fraction to verify each contest
contest_totals_df = vote_count_df.groupby(["contest","vote"])["num_votes"].sum().reset_index()
sample_fractions = []
for i in range(len(contests)):
#construct a vector with N_w 1s, N_l 0s, and N_u 1/2s
#N_w is the number of votes for the winner, N_l is the number of votes for the loser who almost one
    valid_votes = sorted(contest_totals_df["num_votes"][(contest_totals_df["contest"] == contests[i]) & (contest_totals_df["vote"] != "NA")].tolist(), reverse = True)
    ballots_cast = sum(contest_totals_df["num_votes"][contest_totals_df["contest"] == contests[i]])
    reported_votes = np.concatenate(
        (np.repeat(0, valid_votes[1]),
         np.repeat(1/2, ballots_cast - valid_votes[0] - valid_votes[1]),
         np.repeat(1, valid_votes[0]))
        )
    sample_fractions.append(get_sample_size_kelly(reported_votes, alpha = .05) / ballots_cast)
   

   

TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'